In [1]:
# all imports needed in the project 
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from skimage import io, color
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.layers import Dense, LSTM, Flatten, Activation, Input, Lambda
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import shap
from skimage.segmentation import slic
from keras.models import load_model
import matplotlib.pyplot as plt

In [52]:
# load the data and preform preprocessing
def load_and_preprocess_images(base_dir):
    images = []
    labels = []
    img_paths = []

    for label, class_dir in enumerate(os.listdir(base_dir)):
        class_path = os.path.join(base_dir, class_dir)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img_paths.append(img_path)
            img = load_img(img_path, target_size=(75, 75))
            img = img_to_array(img)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # Convert to BGR color space
            img = img.astype(np.uint8)  # Ensure the image is of type CV_8UC3
            img = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)  # Denoise
            img = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)  # Convert to Lab color space
            images.append(img)
            labels.append(label)

    return np.array(images), np.array(labels), img_paths



In [3]:
#Fetch the data
train_dir = "data/train1"
X_train, y_train, img_paths_train = load_and_preprocess_images(train_dir)

In [4]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val, img_paths_train, img_paths_val = train_test_split(X_train, y_train, img_paths_train, test_size=0.1, random_state=42)

In [5]:
# Feature Extraction: Use a pre-trained model (InceptionV3)
inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(75, 75, 3))
X_train_features = inception_model.predict(preprocess_input(X_train))
X_val_features = inception_model.predict(preprocess_input(X_val))


66/66 [==============================] - 11s 170ms/step


In [6]:
# Reshape the features for LSTM
X_train_features_reshaped = X_train_features.reshape(X_train_features.shape[0], -1)
X_val_features_reshaped = X_val_features.reshape(X_val_features.shape[0], -1)

In [101]:
# Build CNN model
cnn_model = Sequential([
    Flatten(input_shape=(1, 1, 2048)),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_features, y_train, epochs=10, batch_size=32, validation_data=(X_val_features, y_val))

# Save CNN model
cnn_model.save("cnn_model.h5")

Epoch 1/10
587/587 [==============================] - 5s 6ms/step - loss: 1.2911 - accuracy: 0.4407 - val_loss: 1.1288 - val_accuracy: 0.5156
Epoch 2/10
587/587 [==============================] - 3s 6ms/step - loss: 0.9653 - accuracy: 0.6026 - val_loss: 0.9732 - val_accuracy: 0.5688
Epoch 3/10
587/587 [==============================] - 4s 6ms/step - loss: 0.7935 - accuracy: 0.6814 - val_loss: 0.7868 - val_accuracy: 0.6974
Epoch 4/10
587/587 [==============================] - 3s 5ms/step - loss: 0.6944 - accuracy: 0.7258 - val_loss: 0.7431 - val_accuracy: 0.7185
Epoch 5/10
587/587 [==============================] - 3s 5ms/step - loss: 0.6255 - accuracy: 0.7537 - val_loss: 0.6585 - val_accuracy: 0.7573
Epoch 6/10
587/587 [==============================] - 3s 6ms/step - loss: 0.5726 - accuracy: 0.7737 - val_loss: 0.6020 - val_accuracy: 0.7746
Epoch 7/10
587/587 [==============================] - 3s 5ms/step - loss: 0.5365 - accuracy: 0.7900 - val_loss: 0.5573 - val_accuracy: 0.8005
Epoch 

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Flatten

# Flatten the data
X_train_features_flattened = X_train_features.reshape((X_train_features.shape[0], -1, X_train_features.shape[-1]))

# Define the RNN model
rnn_model = Sequential([
    SimpleRNN(64, input_shape=(X_train_features_flattened.shape[1], X_train_features_flattened.shape[2])),
    Dense(5, activation='softmax')
])

rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
rnn_model.fit(X_train_features_flattened, y_train, epochs=10, validation_split=0.2)

# Save RNN model
rnn_model.save("rnn_model.h5")


Epoch 1/10
469/469 [==============================] - 3s 5ms/step - loss: 1.3321 - accuracy: 0.4146 - val_loss: 1.0999 - val_accuracy: 0.5602
Epoch 2/10
469/469 [==============================] - 2s 4ms/step - loss: 1.0063 - accuracy: 0.5848 - val_loss: 0.8801 - val_accuracy: 0.6450
Epoch 3/10
469/469 [==============================] - 2s 4ms/step - loss: 0.8169 - accuracy: 0.6720 - val_loss: 0.7950 - val_accuracy: 0.6812
Epoch 4/10
469/469 [==============================] - 2s 4ms/step - loss: 0.7007 - accuracy: 0.7249 - val_loss: 0.6599 - val_accuracy: 0.7441
Epoch 5/10
469/469 [==============================] - 2s 4ms/step - loss: 0.6158 - accuracy: 0.7639 - val_loss: 0.6377 - val_accuracy: 0.7455
Epoch 6/10
469/469 [==============================] - 2s 4ms/step - loss: 0.5600 - accuracy: 0.7802 - val_loss: 0.6117 - val_accuracy: 0.7391
Epoch 7/10
469/469 [==============================] - 2s 4ms/step - loss: 0.5084 - accuracy: 0.8078 - val_loss: 0.5685 - val_accuracy: 0.7729
Epoch 

In [13]:
# Reshape the data for RandomForestClassifier
X_train_features_reshaped = X_train_features_reshaped.reshape((X_train_features_reshaped.shape[0], -1))

# Build RandomForest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_features_reshaped, y_train)

# Save RandomForest model
import joblib
joblib.dump(rf_model, "random_forest_model.joblib")


['random_forest_model.joblib']

In [84]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# cnn evaluate model
def evaluate_model(model, X, y, model_name):
    y_pred_prob = model.predict(X)
    y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class predictions
    accuracy = accuracy_score(y, y_pred)
    conf_matrix = confusion_matrix(y, y_pred)
    classification_report_str = classification_report(y, y_pred)

    print(f"Evaluation results for {model_name}:\n")
    print(f"Accuracy: {accuracy:.4f}\n")
    print("Confusion Matrix:")
    print(conf_matrix)
    print("\nClassification Report:")
    print(classification_report_str)
    return accuracy

# Example usage:
cnn_accuracy = evaluate_model(cnn_model, X_val_features, y_val, "CNN")


66/66 [==============================] - 0s 3ms/step
Evaluation results for CNN:

Accuracy: 0.8125

Confusion Matrix:
[[308  14  44  47  14]
 [  4 432  10  11  24]
 [ 12   1 272  85   1]
 [ 13   5  84 297   4]
 [  7   8   3   0 385]]

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.72      0.80       427
           1       0.94      0.90      0.92       481
           2       0.66      0.73      0.69       371
           3       0.68      0.74      0.70       403
           4       0.90      0.96      0.93       403

    accuracy                           0.81      2085
   macro avg       0.81      0.81      0.81      2085
weighted avg       0.82      0.81      0.81      2085



In [86]:
# Save outputs to a CSV file
X_val_features_reshapedRnn = np.expand_dims(X_val_features, axis=1)

# Ensure the shape matches the expected input shape of the RNN model
X_val_features_reshapedRnn = X_val_features_reshapedRnn.reshape((X_val_features_reshaped.shape[0], 1, -1))
cnn_predictions = cnn_model.predict(X_val_features)
lstm_predictions = rnn_model.predict(X_val_features_reshapedRnn)
rf_predictions = rf_model.predict(X_val_features_2d)

66/66 [==============================] - 0s 4ms/step


In [87]:
#save the data into a csv file 
df = pd.DataFrame({
    'Image_Path': img_paths_val,
    'True_Label': y_val,
    'CNN_Prediction': np.argmax(cnn_predictions, axis=1),
    'RNN_Prediction': np.argmax(lstm_predictions, axis=1),
    'RandomForest_Prediction': rf_predictions,
})

df.to_csv('blood_stain_recognition_results.csv', index=False)